In [1]:
import dask
import dask.dataframe as dd
from scipy import stats
import os
import sys
import pandas as pd
import subprocess as sp
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import shutil
import glob
import gimmemotifs
from pathlib import Path
import qnorm
from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, average_precision_score

%matplotlib inline

In [2]:
def hue_regplot(data, x, y, hue, palette=None, **kwargs):
    from matplotlib.cm import get_cmap
    
    regplots = []
    
    levels = data[hue].unique()
    
    if palette is None:
        default_colors = get_cmap('tab10')
        palette = {k: default_colors(i) for i, k in enumerate(levels)}
    
    for key in levels:
        regplots.append(
            sns.regplot(
                x=x,
                y=y,
                fit_reg=False, 
                data=data[data[hue] == key],
                color=palette[key],
                **kwargs
            )
        )
    
    return regplots

In [3]:
output_dir = '/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3'
Path(f"{output_dir}").mkdir(parents=True, exist_ok=True)       

sample_data_file = '/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/Cell_types_files_v2.csv'
#load the genome info
genome_path_size = "/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/genome/hg38/hg38.fa.sizes"
genome_path = "/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/genome/hg38/hg38.fa"
#genome_path_size = "/ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/Ananse_test_data/genomes/GRCh38.p13/GRCh38.p13.fa.sizes"
#genome_path = "/ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/Ananse_test_data/genomes/GRCh38.p13/GRCh38.p13.fa"

In [4]:
#Lets loop over all the comparisons Ananse needs to make:
sample_data = pd.read_table(sample_data_file, 
                            sep = ',', comment = '#')
#sample_data = sample_data.iloc[[0,1],:]

#sample_data=sample_data[3:10] use this for smaller populations comparison

sample_data

,cell_type,Accesibility_peakfiles,Merged_peakfiles,scATAC_BAMfiles,TPM_matrix,compare_with,count_table_files,count_table_files_blob
0,CB,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupy...,CB_blobun,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/bl...
1,CjS,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupy...,CjS_blobun,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/bl...
2,CSB,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupy...,CSB_blobun,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/bl...
3,CSSCs,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupy...,CSSCs_blobun,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/bl...
4,LNPCs,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupy...,LNPCs_blobun,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/bl...
5,LPCs,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupy...,LPCs_blobun,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/bl...
6,StC,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupy...,StC_blobun,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/bl...
7,MEC,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupy...,MEC_blobun,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/bl...
8,CB_blobun,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupy...,MEC,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/bl...,NaN
9,CjS_blobun,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data...,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupy...,MEC,/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/bl...,NaN


# Ananse binding

In [12]:
#lets first perform motif enrichment
if not os.path.exists(f"{output_dir}scan_results_acc.scanfile"): 
    sp.check_call(f'nice -15 gimme scan '
        f'-T -N 4 '
        f'-g hg38 '
        f'/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/merge_peaks/2021-27-06/tmp/accesible_summits.bed '
        f'> {output_dir}scan_results_acc.scanfile',shell = True)   

In [ ]:
#lets perform motif enrichment for ESC data
if not os.path.exists(f"{output_dir}scan_results_ESC.scanfile"): 
    sp.check_call(f'nice -15 gimme scan '
        f'-T -N 4 '
        f'-g hg38 '
        f'/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/ESC_ANANSE/all_peaks.tsv '
        f'> {output_dir}scan_results_ESC.scanfile',shell = True)

In [ ]:
# lets do ananse binding for your cell populations
i = 0
for index, cell_type in sample_data.iterrows():
    cell_id = sample_data.iloc[index,0]
    print(cell_id)
    Path(f"{output_dir}/{cell_id}").mkdir(parents=True, exist_ok=True)       
    bam_file = sample_data.iloc[index,3]
    
    if not os.path.exists(f"{output_dir}/{cell_id}/binding.h5"): 
        print(f'running ananse binding for celltype {cell_id} using the scATAC bamfile {bam_file} with the ATAC merged peaks of all joined peaks')
        #with open(f'{output_dir}/{cell_id}/myfile.txt', 'w') as fp: 
       #     pass
        sp.check_call(f'ananse binding '
                f'-g {genome_path} '
                f'-A {bam_file} '
                f'-r /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/merge_peaks/accesible_summits.bed '
                f'-o {output_dir}/{cell_id} '
                f'--pfmscorefile /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outsscan_results_acc.scanfile '
                f'-n 4 '    
                f'2> {output_dir}/{cell_id}/ananse_binding_log.txt',shell = True)

CB
CjS
CSB
CSSCs
LNPCs
LPCs
StC
MEC
CB_blobun
running ananse binding for celltype CB_blobun using the scATAC bamfile /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/split_bam_blob/splitblob/outs/subsets/CB_blob.bam with the ATAC merged peaks of all joined peaks
CjS_blobun
running ananse binding for celltype CjS_blobun using the scATAC bamfile /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/split_bam_blob/splitblob/outs/subsets/CjS_blob.bam with the ATAC merged peaks of all joined peaks
CSB_blobun
running ananse binding for celltype CSB_blobun using the scATAC bamfile /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/split_bam_blob/splitblob/outs/subsets/CSB_blob.bam with the ATAC merged peaks of all joined peaks
CSSCs_blobun
running ananse binding for celltype CSSCs_blobun using the scATAC bamfile /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/split_bam_blob/splitblob/outs/subsets/CSSCs_blob.bam with the ATAC merged peaks of all joined peaks
LNPCs_blobun


In [18]:
# generating the binding.h5 file for ESC data ananse 0.3.0
Path(f"{output_dir}/ESC").mkdir(parents=True, exist_ok=True)       
if not os.path.exists(f"{output_dir}/ESC/binding.h5"): 
    sp.check_call(f'ananse binding '
        f'-g {genome_path} '
        f'-A /ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/Ananse_test_data/H3K27ac/results/final_bam/hg38-GSM466732.samtools-coordinate.bam '
        f'-r /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/ESC_ANANSE/all_peaks.tsv '
        f'-o /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3/ESC '
        f'--pfmscorefile /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3scan_results_ESC.scanfile '
        f'-n 4 '   
        f'2> {output_dir}/ESC/ananse_binding_log.txt',shell = True)

# Ananse network

Generate GRN's based on the binding.tsv output and RNAseq expression

In [7]:
#make one TPM file per sample using the big count matrix containing all samples; already done
# ensembl2hugo =  pd.read_table('/ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/genomepy_genomes/GRCh38.p13/gene_id2name.tsv',sep = '\t', comment ='_',names = ['ensembl','HUGO'])
# for index, cell_type in sample_data.iterrows():
#     for binding_mode in ['only_H3K27ac','Remap_H3K27ac']:
#         cell_id = sample_data.iloc[index,0]
#         TPM_matrix_file = sample_data.TPM_matrix[index]
#         TPM_matrix = pd.read_table(TPM_matrix_file,sep = '\t',index_col = 0)
        
#         #convert matrix to gene name; not nessesary if you do it beforehand
#         #hugo_tpm_matrix = TPM_matrix.merge(ensembl2hugo, left_index = True, right_on = 'ensembl', how = 'inner')
#         #hugo_tpm_matrix.index = hugo_tpm_matrix.HUGO
#         #hugo_tpm_matrix = hugo_tpm_matrix.drop_duplicates()
#         #hugo_tpm_matrix = hugo_tpm_matrix.drop_duplicates(subset='HUGO', keep="first")
#         #hugo_tpm_matrix = hugo_tpm_matrix.drop(['ensembl','HUGO'], axis = 1)

#         TPM_files = sample_data.TPM_names[index].split(' ')
#         Path(f"{output_dir}/{binding_mode}/{cell_id}/TPM_files/").mkdir(parents=True, exist_ok=True) 
#         for sample in TPM_files:
#             sample_matrix = hugo_tpm_matrix[sample]
#             sample_matrix = pd.DataFrame(sample_matrix)
#             sample_matrix[['target_id']] = sample_matrix.index
#             sample_matrix[['tpm']] = sample_matrix[[sample]]
#             sample_matrix[['target_id','tpm']]
#             #save sample TPM file
#             sample_matrix[['target_id','tpm']].to_csv(f"{output_dir}/{binding_mode}/{cell_id}/TPM_files/FPM_{sample}.tsv", sep = '\t', index = False, header = True)


In [8]:
#hugo_tpm_matrix

,control_t0B,control_t0A,tcdd_t2A,tcdd_t2B,control_t24A,control_t24B,tcdd_t24A,tcdd_t24B
HUGO,,,,,,,,
TSPAN6,17.149720,10.471415,10.595141,7.870116,6.327820,7.063620,7.979823,6.325595
TNMD,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DPM1,142.842093,107.685809,69.834197,68.042334,55.006878,53.782104,49.323561,41.976615
SCYL3,4.579877,3.434859,2.408801,1.757298,2.696829,2.355813,2.560131,2.280325
C1orf112,24.530547,17.480404,16.068732,11.890818,1.880808,1.570157,6.456277,6.035803
...,...,...,...,...,...,...,...,...
AC064824.1,0.056404,0.107946,0.085177,0.021252,0.000000,0.016335,0.000000,0.032467
AL136225.2,0.056168,0.135234,0.090534,0.037190,0.000000,0.000000,0.115257,0.082025
AC004636.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [48]:
for index, cell_type in sample_data.iterrows():
    cell_id = sample_data.iloc[index,0]
    print(cell_id)
    binding_file = f"{output_dir}/{cell_id}/binding.h5"
    print(f'running ananse binding using the enhancer file {binding_file}')
    file_name = sample_data.iloc[index,4]
    print(f'together with the TPM file {file_name}')
    print(f'and the output file full_network_includeprom.txt')
    
    if not os.path.exists(f"{output_dir}/{cell_id}/full_network_includeprom.txt"): 
        sp.check_call(f'nice -15 ananse network '
            f'-e {file_name} '
            f'-b {binding_file} '
            f'-o {output_dir}/{cell_id}/full_network_includeprom.txt '
            f'-g hg38 '
            f'-n 1 '
            f'2> {output_dir}/{cell_id}/ananse_network_log.txt',shell = True)

CB
running ananse binding using the enhancer file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3/CB/binding.h5
together with the TPM file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupyter_notebooks/CBtpm.tsv
and the output file full_network_includeprom.txt
CjS
running ananse binding using the enhancer file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3/CjS/binding.h5
together with the TPM file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupyter_notebooks/CjStpm.tsv
and the output file full_network_includeprom.txt
CSB
running ananse binding using the enhancer file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3/CSB/binding.h5
together with the TPM file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupyter_notebooks/CSBtpm.tsv
and the output file full_network_includeprom.txt
CSSCs
running ananse binding using the enhancer file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3/CSSCs/binding.h5
together with the 

In [19]:
# generate the ESC network file from the binding.h5 file
binding_file = f"{output_dir}/ESC/binding.h5"
print(f'running ananse binding using the enhancer file {binding_file}')
file_name = "/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/ESC_ANANSE/tpm_ESC.tsv"
print(f'together with the TPM file {file_name}')
print(f'and the output file full_network_includeprom.txt')
    
if not os.path.exists(f"{output_dir}/ESC/full_network_includeprom.txt"): 
    sp.check_call(f'nice -15 ananse network '
        f'-e {file_name} '
        f'-b {binding_file} '
        f'-o /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3/ESC/full_network_includeprom.txt '
        f'-g hg38 '
        f'-n 1 '
        f'2> {output_dir}/ESC/ananse_network_log.txt',shell = True)

running ananse binding using the enhancer file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3/ESC/binding.h5
together with the TPM file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/ESC_ANANSE/tpm_ESC.tsv
and the output file full_network_includeprom.txt


# Ananse Influence

Lets finally run Ananse influence to predict key TFs that differ between cell types.
it will compare each network with the networks listed in the 'comparison' column

In [6]:
# check threading and os delay
import joblib
from joblib import Parallel, delayed
from joblib import parallel_backend

print(Parallel(n_jobs=2)(delayed(os.getpid)() for i in range(4)))

with parallel_backend('threading'):
    print(Parallel(n_jobs=2)(delayed(os.getpid)() for i in range(4)))

[43334, 43335, 43334, 43334]
[42583, 42583, 42583, 42583]


In [8]:
# adapting the recursion limit
print(sys.getrecursionlimit())
sys.setrecursionlimit(100000)
print(sys.getrecursionlimit())

10000
100000


In [9]:
# cell population of interest against only ESC data
for index, cell_type in sample_data.iterrows():
    cell_id = sample_data.iloc[index,0]
    print(cell_id)
    network_file = f"{output_dir}/{cell_id}/full_network_includeprom.txt"
    print(f'running ananse influence using the network file {network_file}')
    net2 = "/ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/Ananse_test_data/analysis/peakpred_V4/only_ATAC/ESCs/full_network_include-promoter.txt"
    print(f'compared to the network file {net2}')
    Path(f"{output_dir}/{cell_id}/ESC{cell_id}_influence_500000_V3").mkdir(parents=True, exist_ok=True) 
    file_name = sample_data.iloc[index,6]
    print(f'together with DEG file {file_name} and the output file influence.txt')

    if not os.path.exists(f"{output_dir}/{cell_id}/ESC{cell_id}_influence_500000_V3/influence.txt"):
        sp.check_call(f'nice -15 ananse influence '
            f'-s {net2} '
            f'-t {network_file} '
            f'-d {file_name} '
            f'-i 500000 '
            f'--plot '    
            f'-o {output_dir}/{cell_id}/ESC{cell_id}_influence_500000_V3/influence.txt '
            f'-n 1 '
            f'2> {output_dir}/{cell_id}/ananse_influence_log_V2.txt',shell = True)

CB
running ananse influence using the network file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3/CB/full_network_includeprom.txt
compared to the network file /ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/Ananse_test_data/analysis/peakpred_V4/only_ATAC/ESCs/full_network_include-promoter.txt
together with DEG file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/20210804/CBESCpseudobulkpadj.tsv and the output file influence.txt


KeyboardInterrupt: 

In [ ]:
# Compare cell_types against blob of all other cells (normalized expression)
for index, cell_type in sample_data.iterrows(): 
    cell_id = sample_data.iloc[index,0]
    comparisons = sample_data.compare_with[index]
    if not cell_id.find("blobun") != -1:
        #print(cell_id)
        #print(comparisons)
        network_file = f"{output_dir}/{cell_id}/full_network_includeprom.txt"
        print(f'running ananse influence using the network file {network_file}')
        net2 = f"{output_dir}/{comparisons}/full_network_includeprom.txt"
        print(f'compared to the network file {net2}')
        Path(f"{output_dir}/{comparisons}/{comparisons}_influence_500000_V3").mkdir(parents=True, exist_ok=True) 
        file_name = sample_data.iloc[index,7]
        print(f'together with DEG file {file_name} and the output file influence.txt')

        if not os.path.exists(f"{output_dir}/{comparisons}/{comparisons}_influence_500000_V3/influence_diffnetwork.txt"):
            sp.check_call(f'nice -15 ananse influence '
                f'-s {net2} '
                f'-t {network_file} '
                f'-d {file_name} '
                f'-i 500000 '
                f'--plot '    
                f'-o {output_dir}/{comparisons}/{comparisons}_influence_500000_V3/influence.txt '
                f'-n 1 '
                f'2> {output_dir}/{comparisons}/ananse_influence_log_V3_500.txt',shell = True)

running ananse influence using the network file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3/CB/full_network_includeprom.txt
compared to the network file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3/CB_blobun/full_network_includeprom.txt
together with DEG file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/blob20210804/CBblobpseudobulkpadj.tsv and the output file influence.txt
running ananse influence using the network file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3/CjS/full_network_includeprom.txt
compared to the network file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs3/CjS_blobun/full_network_includeprom.txt
together with DEG file /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/R/blob20210804/CjSblobpseudobulkpadj.tsv and the output file influence.txt


In [ ]:
# extra code    
    comparisons = sample_data.compare_with[index]
        cell_id = sample_data.iloc[index,0]
        #get the count data files for the source cell type
        #count_files_source_cell_type = sample_data.TPM_names[index].split(' ')
        if not comparisons != comparisons: #do stuff if there is a string in the comparisons column
            for comparison in comparisons.split(' '):
                #get the count data files for the target cell type
                target_data = sample_data.loc[sample_data['cell_type'] == comparison]
                target_data = target_data.reset_index(drop=True)
                count_files_target_cell_type = target_data.TPM_names[0].split(' ')
                #lets run deseq between the sample comparisons 
                comp_name = f'{cell_id}_to_{comparison}'
                print(f'comparing {comp_name}')
                if not os.path.exists(f"{output_dir}/{binding_mode}/influence/{comp_name}"):
                    Path(f"{output_dir}/{binding_mode}/influence/{comp_name}").mkdir(parents=True, exist_ok=True)
                    
                        source_network = f'{output_dir}/{cell_id}/full_network__includeprom.txt'
                        target_network = f'{output_dir}/{comparison}/full_network_includeprom.txt'
                        influence_output_name = f'{output_dir}/{binding_mode}/influence/{comp_name}/{weight_modus}/{n_edges}/'
                        if not os.path.exists(f"{influence_output_name}"):
                            Path(f"{influence_output_name}").mkdir(parents=True, exist_ok=True)
                        if not os.path.exists(f"{influence_output_name}influence.txt"):
                            print(f'running influence for {comp_name} with the datatype of {binding_mode} and {weight_modus}')
                            sp.check_call(f'nice -15 ananse influence '
                                f'-s {source_network} '
                                f'-t {target_network} '
                                f'-d {DEG_file} '
                                f'--edges {n_edges} '
                                f'-p '
                                f'-o {influence_output_name}influence.txt ' 
                                f'-n 1 '
                                f'2> {influence_output_name}log.txt',shell = True)   

In [ ]:
### delete in full??

ensembl2hugo =  pd.read_table('/ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/genomepy_genomes/GRCh38.p13/gene_id2name.tsv',sep = '\t', comment ='_',names = ['ensembl','HUGO'])



nice -15 ananse influence 
-s /ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/Ananse_test_data/analysis/peakpred_V4/only_ATAC/ESCs/full_network_include-promoter.txt 
-t /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs/CjS/full_network_includeprom.txt 
-d /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/pseudobulk_rnaseq/CjSESC_degenes.tsv 
-o /ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/ANANSE/outs/CjS/ESCCjS_influence/influence.txt -n 10


for index, cell_type in sample_data.iterrows():
    for binding_mode in ['Remap_H3K27ac','only_H3K27ac']:
        comparisons = sample_data.compare_with[index]
        cell_id = sample_data.iloc[index,0]
        #get the count data files for the source cell type
        count_files_source_cell_type = sample_data.TPM_names[index].split(' ')
        if not comparisons != comparisons: #do stuff if there is a string in the comparisons column
            for comparison in comparisons.split(' '):
                #get the count data files for the target cell type
                target_data = sample_data.loc[sample_data['cell_type'] == comparison]
                target_data = target_data.reset_index(drop=True)
                count_files_target_cell_type = target_data.TPM_names[0].split(' ')
                #lets run deseq between the sample comparisons 
                comp_name = f'{cell_id}_to_{comparison}'
                print(f'comparing {comp_name}')
                if not os.path.exists(f"{output_dir}/{binding_mode}/influence/{comp_name}"):
                    Path(f"{output_dir}/{binding_mode}/influence/{comp_name}").mkdir(parents=True, exist_ok=True)

                #lets load the countmatrix, and remove all columns that don't contain one of the samples
                count_matrix = pd.read_table(sample_data.count_table_files[index])
                #count_matrix[(count_files_source_cell_type + count_files_target_cell_type)]
                #switch ensembl to HUGO gene names
                hugo_count_matrix = count_matrix.merge(ensembl2hugo, left_on = 'gene', right_on = 'ensembl', how = 'inner')
                hugo_count_matrix.index = hugo_count_matrix.HUGO
                hugo_count_matrix = hugo_count_matrix.drop_duplicates()
                hugo_count_matrix = hugo_count_matrix.drop_duplicates(subset='HUGO', keep="first")
                hugo_count_matrix = hugo_count_matrix.drop(['ensembl','HUGO','gene'], axis = 1)
                #hugo_count_matrix.to_csv(f"{output_dir}/{binding_mode}/influence/{comp_name}/count_matrix.tsv", sep = '\t', index = True, header = True)
                hugo_count_matrix = hugo_count_matrix[(count_files_source_cell_type + count_files_target_cell_type)]
                hugo_count_matrix.to_csv(f"{output_dir}/{binding_mode}/influence/{comp_name}/count_matrix.tsv", sep = '\t', index = True, header = True)
                #make a sample file for deseq2 
                sample_names = (count_files_source_cell_type + count_files_target_cell_type)
                annotation = ["source"] * len(count_files_source_cell_type) + ["target"] * len(count_files_target_cell_type)
                data= {'samples':sample_names, 'condition':annotation}
                sample_info_df = pd.DataFrame(data)
                sample_info_df.to_csv(f"{output_dir}/{binding_mode}/influence/{comp_name}/sample_info.tsv", sep = ',', index = False, header = True)
                #run Deseq2 
                #if not os.path.exists(f"{output_dir}/{binding_mode}/influence/{comp_name}/DEG.tsv"):
                
                if not os.path.exists(f"{output_dir}/{binding_mode}/influence/{comp_name}/DEGs.csv"):
                    print('running Deseq2')
                    sp.check_call(
                        f'nice -5 Rscript --vanilla '
                        f'/ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/Ananse_test_data/analysis/RunDeseq2.R '
                        f'{output_dir}/{binding_mode}/influence/{comp_name}/count_matrix.tsv '
                        f'{output_dir}/{binding_mode}/influence/{comp_name}/sample_info.tsv '
                        f'{output_dir}/{binding_mode}/influence/{comp_name}/DEGs.csv '
                        f'2> {output_dir}/{binding_mode}/influence/{comp_name}/deseq2log.txt',
                        shell = True)

                    DEG_table = pd.read_table(f'{output_dir}/{binding_mode}/influence/{comp_name}/DEGs.csv', 
                            sep = ',', comment = '#')
                    DEG_table['resid'] = DEG_table['region']
                    DEG_table = DEG_table.dropna()
                    DEG_table[['resid','log2FoldChange','padj']].to_csv(f'{output_dir}/{binding_mode}/influence/{comp_name}/DEG.tsv', sep = '\t', index = False, header = True)
                    DEG_file = f'{output_dir}/{binding_mode}/influence/{comp_name}/DEG.tsv'

                for weight_modus in ['include-promoter']:
                    for n_edges in [100000, 500000]:
                        source_network = f'{output_dir}/{binding_mode}/{cell_id}/full_network_{weight_modus}.txt'
                        target_network = f'{output_dir}/{binding_mode}/{comparison}/full_network_{weight_modus}.txt'
                        influence_output_name = f'{output_dir}/{binding_mode}/influence/{comp_name}/{weight_modus}/{n_edges}/'
                        if not os.path.exists(f"{influence_output_name}"):
                            Path(f"{influence_output_name}").mkdir(parents=True, exist_ok=True)
                        if not os.path.exists(f"{influence_output_name}influence.txt"):
                            print(f'running influence for {comp_name} with the datatype of {binding_mode} and {weight_modus}')
                            sp.check_call(f'nice -15 ananse influence '
                                f'-s {source_network} '
                                f'-t {target_network} '
                                f'-d {DEG_file} '
                                f'-p '
                                f'-o {influence_output_name}influence.txt ' 
                                f'--edges {n_edges} '
                                f'-n 12 '
                                f'2> {influence_output_name}log.txt',shell = True)  

comparing KC_0h_CTR_to_KC_TCDD30_2h_RNA
running influence for KC_0h_CTR_to_KC_TCDD30_2h_RNA with the datatype of Remap_H3K27ac and include-promoter
comparing KC_0h_CTR_to_KC_TCDD90_2h_RNA
running Deseq2
running influence for KC_0h_CTR_to_KC_TCDD90_2h_RNA with the datatype of Remap_H3K27ac and include-promoter
running influence for KC_0h_CTR_to_KC_TCDD90_2h_RNA with the datatype of Remap_H3K27ac and include-promoter
comparing KC_0h_CTR_to_KC_TCDD90_24h_RNA
running Deseq2
running influence for KC_0h_CTR_to_KC_TCDD90_24h_RNA with the datatype of Remap_H3K27ac and include-promoter
running influence for KC_0h_CTR_to_KC_TCDD90_24h_RNA with the datatype of Remap_H3K27ac and include-promoter
comparing KC_0h_CTR_to_KC_TCDD30_2h_RNA
running Deseq2
running influence for KC_0h_CTR_to_KC_TCDD30_2h_RNA with the datatype of only_H3K27ac and include-promoter
running influence for KC_0h_CTR_to_KC_TCDD30_2h_RNA with the datatype of only_H3K27ac and include-promoter
comparing KC_0h_CTR_to_KC_TCDD90_2h_RN